# 穩定可以執行的tkinter錄影程式
# 可以開啟相機至少700秒以上而且視窗錄影的

In [4]:
import tkinter as tk
import cv2
from PIL import Image, ImageTk, Image
import numpy as np
import face_recognition
import random
import time

# 預先載入加分&扣分物的圖片
obj_width = 40

garbage_img = cv2.imread('apple.jpeg')
garbage_img = cv2.resize(garbage_img,(40,40),interpolation=cv2.INTER_CUBIC)
garbage_rows, garbage_cols, channels = garbage_img.shape

present_img = cv2.imread('bad.png')
present_img = cv2.resize(present_img,(40,40),interpolation=cv2.INTER_CUBIC)
present_rows, present_cols, channels = present_img.shape

print(garbage_rows, garbage_cols, channels)
print(present_rows, present_cols, channels)


40 40 3
40 40 3


In [ ]:
root = tk.Tk()

# Load a sample picture and learn how to recognize it.
obama_image = face_recognition.load_image_file("obama.jpg")
obama_face_encoding = face_recognition.face_encodings(obama_image)[0]
zonejia_image = face_recognition.load_image_file("zonejia_image.jpg")
zonejia_face_encoding = face_recognition.face_encodings(zonejia_image)[0]

# 當等於True才處理該Frame，False則忽略
process_this_frame = True

# 單純攝影機設定
width, height = 1000, 750
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
face_landmarks_list = []
# ------------------遊戲初始區----------------------
# 初始化 加分掉落物
init_y = 5
garbage_location_x = np.random.randint(low=4, high=width-obj_width, size=5)
garbage_location_y = np.zeros(5)*10
# 初始化 扣分掉落物
present_location_x = np.random.randint(low=4, high=width-obj_width, size=5)
present_location_y = np.zeros(5)*10

# 倒數計時出現的加分數字(位置)
bonus_position = (0,0) # (x, y)

# 每隔10毫秒掉落距離
garbage_speed = 5
# 分數
score = 0
# ------------------遊戲初始區----------------------

tStart = -1
tEnd = -1
finalCount = 0
def peace_close():
    global root, cap
#     root.quit()
    root.destroy() 
    cap.release()
    
root.bind('<Escape>', lambda e: peace_close)
lmain = tk.Label(root)
lmain.pack()

close_btn = tk.Button(root, text='點一下關閉', command=peace_close)
close_btn.pack()

def show_frame():
    # Initialize some variables
    global face_locations, face_encodings, face_names
    
    global process_this_frame
    global garbage_location_x, garbage_location_y
    global present_location_x, present_location_y

    global score, tStart, tEnd, bonus_position, finalCount
    _, frame = cap.read()
    frame = cv2.flip(frame, 1) # 將照片左右轉向一下

    
    
    # 畫圖開始
#     small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    
    small_frame = cv2.resize(frame, (0, 0), fx=0.5, fy=0.5)
    
    # 計時
    if tStart < 0:
        tStart = time.time()
    tEnd = time.time()
    finalSecond = int(tEnd - tStart)

    print('目前時間：', finalSecond)
    
    # 繪畫倒數加分
    if finalSecond >= 10:
        if finalCount == (finalSecond-10):
            finalCount += 1
            bonus_position = random.randint(2, width-obj_width*2),random.randint(2, height-obj_width*2)
        if bonus_position != (0,0):
            font = cv2.FONT_HERSHEY_DUPLEX
            # 適合十位數顯示
            cv2.putText(frame, str(finalSecond), bonus_position, font, 1.0, (255, 0, 255))
            x1, y2 = bonus_position
            cv2.rectangle(frame, (x1, y2-obj_width//2), (x1+obj_width, y2), (255, 0, 255))
            
    # Only process every other frame of video to save time
    if process_this_frame:
        # Find all the faces and face encodings in the current frame of video
        face_locations = face_recognition.face_locations(small_frame)
        face_encodings = face_recognition.face_encodings(small_frame, face_locations)
        
        face_names = []
        
        # 找出所有臉中是否有符合 '歐巴馬臉'或'宗嘉臉'的
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            match = face_recognition.compare_faces([obama_face_encoding, zonejia_face_encoding], face_encoding)
            name = "Unknown"

            if match[0]:
                name = "Barack"
            if match[1]:
                name = 'Zone-Jia'

            face_names.append(name)

        
    process_this_frame = not process_this_frame

    
    # Display the results
    for (top, right, bottom, left), name in zip(face_locations, face_names):
        # Scale back up face locations since the frame we detected in was scaled to 1/4 size
#         top *= 4
#         right *= 4
#         bottom *= 4
#         left *= 4
        top *= 2
        right *= 2
        bottom *= 2
        left *= 2

        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

        # Draw a label with a name below the face
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        # 判斷是否撞到 加分的garbage
        for jjj in range(garbage_location_x.size):
            rec_x1, rec_y1 = int(garbage_location_x[jjj]), int(garbage_location_y[jjj])
            rec_x2, rec_y2 = rec_x1+obj_width, rec_y1+obj_width

            #             A.x+A.W >= B.x ： 臉的右邊界大於等於 Sprite-B 的左邊界。
#             A.x <= B.x+B.W ： 臉的左邊界小於等於 Sprite-B 的右邊界。
#             A.y+A.H >= B.y ： Sprite-A 的下邊界大於等於 Sprite-B 的上邊界。
#             A.y <= B.y+B.H ： Sprite-A 的上邊界小於等於 Sprite-B 的下邊界。

            if right >= rec_x1 and left <= rec_x2 and bottom >= rec_y1 and top <= rec_y2:
                score += 1
                # 被撞到的好garbage_location_x[jjj]
                garbage_location_y[jjj] = init_y
                garbage_location_x[jjj] = random.randint(2, width-obj_width)
        # 判斷是否撞到 扣分的present
        for jjj in range(present_location_x.size):
            rec_x1, rec_y1 = int(present_location_x[jjj]), int(present_location_y[jjj])
            rec_x2, rec_y2 = rec_x1+obj_width, rec_y1+obj_width

#             A.x+A.W >= B.x ： 臉的右邊界大於等於 Sprite-B 的左邊界。
#             A.x <= B.x+B.W ： 臉的左邊界小於等於 Sprite-B 的右邊界。
#             A.y+A.H >= B.y ： Sprite-A 的下邊界大於等於 Sprite-B 的上邊界。
#             A.y <= B.y+B.H ： Sprite-A 的上邊界小於等於 Sprite-B 的下邊界。

            if right >= rec_x1 and left <= rec_x2 and bottom >= rec_y1 and top <= rec_y2:
                score -= 1
                # 被撞到的壞present_location_x[jjj]
                present_location_y[jjj] = init_y
                present_location_x[jjj] = random.randint(2, width-obj_width)
                

    # 繪畫掉落物
    garbage_location_y += garbage_speed
    present_location_y += garbage_speed
    
    for jjj in range(garbage_location_x.size):
        rec_x1, rec_y1 = int(garbage_location_x[jjj]), int(garbage_location_y[jjj])
        rec_x2, rec_y2 = rec_x1+obj_width, rec_y1+obj_width
#         # 畫長方形30x30
#         cv2.rectangle(frame, (rec_x1, rec_y1), (rec_x2, rec_y2), (166, 88, 255), 2, cv2.FILLED)
        # 畫圖片一樣30x30
        if rec_y2 >= height or rec_x2 >= width:
            garbage_location_x[jjj] = random.randint(2, width-obj_width)
            garbage_location_y[jjj] = init_y
        frame[rec_y1:rec_y2, rec_x1:rec_x2] = garbage_img
        
    garbage_location_x[garbage_location_y>height] = random.randint(0, width-obj_width)
    garbage_location_y[garbage_location_y>height] = init_y
    
    for jjj in range(present_location_x.size):
        rec_x1, rec_y1 = int(present_location_x[jjj]), int(present_location_y[jjj])
        rec_x2, rec_y2 = rec_x1+obj_width, rec_y1+obj_width
        
#         cv2.rectangle(frame, (rec_x1, rec_y1), (rec_x2, rec_y2), (0, 0, 0), 2)
        # 畫圖片一樣30x30
        if rec_y2 >= height or rec_x2 >= width:
            present_location_x[jjj] = random.randint(2, width-obj_width)
            present_location_y[jjj] = init_y
        frame[rec_y1:rec_y2, rec_x1:rec_x2] = present_img
        
    present_location_x[present_location_y>height] = random.randint(0, width-obj_width)
    present_location_y[present_location_y>height] = init_y
    
    # 繪畫 '分數顯示'
    cv2.putText(frame, 'Score:'+str(score), (10, 45), cv2.FONT_HERSHEY_TRIPLEX, 2.0, (0, 255, 0), 1)
    
    # 畫圖結束
    
    
    cv2image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(cv2image)
    imgtk = ImageTk.PhotoImage(image=img)
    lmain.imgtk = imgtk
    lmain.configure(image=imgtk)
    lmain.after(10, show_frame)

show_frame()
root.mainloop()

cap.release()
print('和平結束')

目前時間： 0
目前時間： 0
目前時間： 0
目前時間： 0
目前時間： 0
目前時間： 0
目前時間： 0
目前時間： 1
目前時間： 1
目前時間： 1
目前時間： 1
目前時間： 1
目前時間： 1
目前時間： 2
目前時間： 2
目前時間： 2
目前時間： 2
目前時間： 2
目前時間： 2
目前時間： 2
目前時間： 2
目前時間： 3
目前時間： 3
目前時間： 3
目前時間： 3
目前時間： 3
目前時間： 3
目前時間： 3
目前時間： 3
目前時間： 4
目前時間： 4
目前時間： 4
目前時間： 4
目前時間： 4
目前時間： 4
目前時間： 5
目前時間： 5
目前時間： 5
目前時間： 5
目前時間： 5
目前時間： 5
目前時間： 5
目前時間： 5
目前時間： 6
目前時間： 6
目前時間： 6
目前時間： 6
目前時間： 6
目前時間： 6
目前時間： 6
目前時間： 7
目前時間： 7
目前時間： 7
目前時間： 7
目前時間： 7
目前時間： 7
目前時間： 7
目前時間： 8
目前時間： 8
目前時間： 8
目前時間： 8
目前時間： 8
目前時間： 8
目前時間： 8
目前時間： 8
目前時間： 9
目前時間： 9
目前時間： 9
目前時間： 9
目前時間： 9
目前時間： 9
目前時間： 9
目前時間： 9
目前時間： 10
目前時間： 10
目前時間： 10
目前時間： 10
目前時間： 10
目前時間： 10
目前時間： 10
目前時間： 10
目前時間： 11
目前時間： 11
目前時間： 11
目前時間： 11
目前時間： 11
目前時間： 11
目前時間： 11
目前時間： 12
目前時間： 12
目前時間： 12
目前時間： 12
目前時間： 12
目前時間： 12
目前時間： 12
目前時間： 13
目前時間： 13
目前時間： 13
目前時間： 13
目前時間： 13
目前時間： 13
目前時間： 13
目前時間： 13
目前時間： 14
目前時間： 14
目前時間： 14
目前時間： 14
目前時間： 14
目前時間： 14
目前時間： 14
目前時間： 14
目前時間： 15
目前時間： 15
目前時間： 15
目前時間： 15
目前時間： 15
目前時間： 15
目前時間： 15
目前時間： 16
目前

In [3]:
# 緊急停止方案
peace_close()

TclError: can't invoke "destroy" command:  application has been destroyed

In [ ]:
a = np.ndarray(shape=(2,2), dtype=float, order='F')
print(a)
print(a*2)

In [ ]:
# 這禮拜要進化該程式碼，加入即使沒有正臉也可以追蹤該人的功能。
# 詳細演算法如下：
    查找該影幀有沒有正臉，有的話將正臉記錄起來。
    查找該影幀紀錄中正臉但未找到的